In [11]:
import torch
import os
import numpy as np
import pandas as pd
import shutil

BASE_DIR = r"D:\understanding_cloud_organization"
torch.cuda.empty_cache()

In [12]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import torch

# Multi-class augmentation
train_transform = A.Compose([
    A.Resize(height=256, width=256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Affine(translate_percent=(0.1, 0.1), scale=(0.9, 1.1), rotate=(-20, 20), p=0.5),  # Corrected Affine
    A.ElasticTransform(p=0.2, alpha=1, sigma=50),  # Removed alpha_affine
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),  # Adjusted for 3 channels
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),  # Adjusted for 3 channels
    ToTensorV2()
])

def load_image_mask(image_path, mask_path, transform):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load multi-class mask
    mask = torch.tensor(mask, dtype=torch.long)  # Convert to tensor
    height, width = 256, 256  # Or any other fixed size
    image = cv2.resize(image, (width, height))
    mask = cv2.resize(mask.numpy(), (width, height), interpolation=cv2.INTER_NEAREST)

    augmented = transform(image=image, mask=mask)  # Apply augmentation
    return augmented["image"], augmented["mask"]

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [13]:
import torch
import os
from torch.utils.data import Dataset, DataLoader

class CloudDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_list = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = self.image_list[idx]
        img_path = os.path.join(self.image_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace(".jpg", ".png"))
        if not os.path.exists(mask_path):
            raise ValueError(f"Mask file not found: {mask_path}")

        image, mask = load_image_mask(img_path, mask_path, self.transform)
        return image, mask

# Create datasets
train_dataset = CloudDataset(
    image_dir=os.path.join(BASE_DIR, "train", "images"),
    mask_dir=os.path.join(BASE_DIR, "train", "masks"),
    transform=train_transform
)

val_dataset = CloudDataset(
    image_dir=os.path.join(BASE_DIR, "val", "images"),
    mask_dir=os.path.join(BASE_DIR, "val", "masks"),
    transform=val_transform
)

test_dataset = CloudDataset(
    image_dir=os.path.join(BASE_DIR, "test", "images"),
    mask_dir=os.path.join(BASE_DIR, "test", "masks"),
    transform=val_transform
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,  pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, pin_memory=True)

print("Multi-class DataLoaders are ready!")


Multi-class DataLoaders are ready!


In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=5):  # 5 classes (background + 4 cloud types)
        super(UNet, self).__init__()

        self.encoder = models.resnet34(weights="IMAGENET1K_V1")
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-2])  # Remove FC layer

        self.conv_up = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.final_conv = nn.Conv2d(256, out_channels, kernel_size=1)  # 5 output channels

    def forward(self, x):
        x = self.encoder(x)
        x = self.upsample(F.relu(self.conv_up(x)))
        x = self.final_conv(x)
        return x  # No sigmoid for multi-class segmentation

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)
print("Multi-class model ready!")

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 203MB/s]


Multi-class model ready!


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import time
from torch.amp import GradScaler
from torch.amp import autocast 

# Define Dice Coefficient function
def dice_coefficient(preds, labels, num_classes=5, smooth=1e-6):
    dice = 0.0
    for c in range(num_classes):
        # Binarize the masks
        pred_c = (preds == c).float()
        label_c = (labels == c).float()

        intersection = (pred_c * label_c).sum()
        union = pred_c.sum() + label_c.sum()

        dice += (2. * intersection + smooth) / (union + smooth)

    return dice / num_classes  # Average over classes

# Loss function & optimizer (unchanged)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
scaler = GradScaler()

# Train model with Dice Coefficient metric
def train_model(model, train_loader, val_loader, epochs):
    model.to(device)  # Move model to the appropriate device
    best_val_loss = float('inf')
    patience = 5
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.train()
        epoch_start_time = time.time()
        running_train_loss = 0.0

        # Training Loop
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]", leave=True)
        for images, masks in progress_bar:
            images, masks = images.to(device), masks.to(device).long()

            optimizer.zero_grad()

            # Mixed precision training
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(images)
                outputs_resized = F.interpolate(outputs, size=(256, 256), mode='bilinear', align_corners=False)
                loss = criterion(outputs_resized, masks)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_train_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        avg_train_loss = running_train_loss / len(train_loader)
        
        # Validation Loop
        model.eval()
        running_val_loss = 0.0
        correct, total = 0, 0
        running_dice = 0.0

        with torch.no_grad():
            progress_bar_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Validation]", leave=True)
            for images, masks in progress_bar_val:
                images, masks = images.to(device), masks.to(device).long()

                outputs = model(images)
                outputs_resized = F.interpolate(outputs, size=(256, 256), mode='bilinear', align_corners=False)
                loss = criterion(outputs_resized, masks)

                running_val_loss += loss.item()

                # Compute accuracy
                preds = torch.argmax(outputs_resized, dim=1)
                correct += (preds == masks).sum().item()
                total += masks.numel()

                # Compute Dice Coefficient
                dice = dice_coefficient(preds, masks, num_classes=5)
                running_dice += dice

                progress_bar_val.set_postfix(loss=loss.item())

        avg_val_loss = running_val_loss / len(val_loader)
        val_accuracy = correct / total * 100
        avg_dice_score = running_dice / len(val_loader)  # Average Dice score over the validation set

        # Update learning rate scheduler
        scheduler.step(avg_val_loss)

        epoch_time = time.time() - epoch_start_time

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            
        print(f"\nEpoch [{epoch+1}/{epochs}] completed in {epoch_time:.2f} sec")
        print(f"  ➤ Train Loss: {avg_train_loss:.4f}")
        print(f"  ➤ Val Loss: {avg_val_loss:.4f}")
        print(f"  ➤ Val Accuracy: {val_accuracy:.2f}%")
        print(f"  ➤ Val Dice Score: {avg_dice_score:.4f}")
        print(f"  ➤ epochs_without_improvement: {epochs_without_improvement}")
        print(f"  ➤ Learning Rate: {optimizer.param_groups[0]['lr']}")
        
# Train model
train_model(model, train_loader, val_loader, epochs=32)

Epoch 1/32 [Validation]: 100%|██████████| 5/5 [00:21<00:00,  4.25s/it, loss=0.993]



Epoch [1/32] completed in 205.60 sec
  ➤ Train Loss: 1.0303
  ➤ Val Loss: 0.9813
  ➤ Val Accuracy: 60.04%
  ➤ Val Dice Score: 0.4953
  ➤ epochs_without_improvement: 0
  ➤ Learning Rate: 0.0001


Epoch 2/32 [Validation]: 100%|██████████| 5/5 [00:21<00:00,  4.24s/it, loss=0.979]



Epoch [2/32] completed in 205.37 sec
  ➤ Train Loss: 0.8548
  ➤ Val Loss: 0.9727
  ➤ Val Accuracy: 60.49%
  ➤ Val Dice Score: 0.5290
  ➤ epochs_without_improvement: 0
  ➤ Learning Rate: 0.0001


Epoch 3/32 [Validation]: 100%|██████████| 5/5 [00:21<00:00,  4.25s/it, loss=0.901]


...


Epoch [30/32] completed in 199.52 sec
  ➤ Train Loss: 0.7256
  ➤ Val Loss: 0.8797
  ➤ Val Accuracy: 65.08%
  ➤ Val Dice Score: 0.5633
  ➤ epochs_without_improvement: 20
  ➤ Learning Rate: 1.0000000000000004e-08


Epoch 31/32 [Validation]: 100%|██████████| 5/5 [00:20<00:00,  4.18s/it, loss=0.894]



Epoch [31/32] completed in 200.55 sec
  ➤ Train Loss: 0.7229
  ➤ Val Loss: 0.8801
  ➤ Val Accuracy: 65.03%
  ➤ Val Dice Score: 0.5638
  ➤ epochs_without_improvement: 21
  ➤ Learning Rate: 1.0000000000000004e-08


Epoch 32/32 [Validation]: 100%|██████████| 5/5 [00:20<00:00,  4.13s/it, loss=0.895]


Epoch [32/32] completed in 201.57 sec
  ➤ Train Loss: 0.7235
  ➤ Val Loss: 0.8804
  ➤ Val Accuracy: 65.09%
  ➤ Val Dice Score: 0.5638
  ➤ epochs_without_improvement: 22
  ➤ Learning Rate: 1.0000000000000004e-08
